<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

Cloning into 'semantic_features_detection'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 1549 (delta 108), reused 60 (delta 34), pack-reused 1392
Receiving objects: 100% (1549/1549), 114.36 MiB | 30.98 MiB/s, done.
Resolving deltas: 100% (960/960), done.


In [ ]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21102205&authkey=APtm68CllMdnF74"  -O kitti.zip

--2020-06-06 16:51:40--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21102205&authkey=APtm68CllMdnF74
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://kvpofq.am.files.1drv.com/y4msz2TE5L2rDhDT3E53oVFKJYlFz4toOBYZLu-e9QvAQu-iUYPjNIhYbEyuxw49e4jiEq3-xEGH7x1HZ0mLjIRCzBqG8laf5XkpjgCtsoLEdky1UFpp3ebcIciQ9iUECUIYiuvPXvEUts2q0xEl7DRn4k4k8O0XAidlzaLlG_bPK6xU5c7oWWCte-xydb610QnVz48owsiV3WqyN0VqpII0A/sequence_00_image_2.zip?download&psid=1 [following]
--2020-06-06 16:51:41--  https://kvpofq.am.files.1drv.com/y4msz2TE5L2rDhDT3E53oVFKJYlFz4toOBYZLu-e9QvAQu-iUYPjNIhYbEyuxw49e4jiEq3-xEGH7x1HZ0mLjIRCzBqG8laf5XkpjgCtsoLEdky1UFpp3ebcIciQ9iUECUIYiuvPXvEUts2q0xEl7DRn4k4k8O0XAidlzaLlG_bPK6xU5c7oWWCte-xydb610QnVz48owsiV3WqyN0VqpII0A/sequence_00_image_2.zip?download&psid=1
Resolving kvpofq.am.files.1drv.com (

In [ ]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21101634&authkey=ABTiTiHVXMI06vU"  -O mapillary_prelim.h5

In [ ]:
!unzip -qq /content/kitti.zip -d /content

In [ ]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install wget

In [ ]:
%tensorflow_version 1.x
%matplotlib inline
import os
import sys
os.chdir('/content/semantic_features_detection/samples/mapillary/cytools')
!python setup.py build_ext --inplace
os.chdir('/content/semantic_features_detection/Notebook')
ROOT_DIR = os.path.abspath('../')
sys.path.append(os.path.join(ROOT_DIR, 'samples'))
import inference

In [ ]:
inf_model = inference.Inference(os.path.join(ROOT_DIR, 'mapillary_prelim.h5'))

In [ ]:
import numpy as np
def masks2instance_im(masks):
    instance_im = np.zeros(shape=[masks.shape[0], masks.shape[1]])
    for i in range(masks.shape[-1]):
        instance_im[masks[...,i]] = (i + 1)
    return instance_im

In [ ]:
import glob
import cv2
IMAGE_DIR_LEFT = '/content/image_2'
IMAGE_DIR_RIGHT = '/content/kitti_dataset/dataset/sequences/00/image_3'
SAVE_DIR = '/content/Results/Instances'
!mkdir /content/Results
!mkdir /content/Results/Instances
!mkdir /content/Results/Visualization

features = {}
features['results'] = []
image_paths = glob.iglob(os.path.join(IMAGE_DIR_LEFT, '*.*'))
for image_path in image_paths:
    feature, masks = inf_model.predict(image_path,save_vis=True, save_dir='/content/Results/Visualization')
    features['results'].append(feature)
    # save instance images
    instance_im =  masks2instance_im(masks)
    image_id=os.path.split(image_path)[1][0:-4]
    cv2.imwrite(os.path.join(SAVE_DIR, 'L' + image_id + '.png'), instance_im)


In [ ]:
import json
FEATURE_FILE = '/content/results.json'
with open(FEATURE_FILE, 'w') as outfile:
    json.dump(features, outfile)
!mv /content/results.json /content/drive/"My Drive"/Kitti_00_Results/results.json

In [ ]:
!mv /content/Results /content/drive/"My Drive"/Kitti_00_Results